# First Graded Assignment 
## (Aaron - Architecture 1)

## Setup

### Init modules and helper functions
Load modules and set seed

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import pandas as pd

# fix random seed for reproducibility
seed = 2021
np.random.seed(seed)  

import sklearn as sk
from sklearn.model_selection import train_test_split

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

Helper function to plot history

In [1]:
#helper functions for visualisation

# same function as in the getting started notebook, 
# but now plotting the loss functions used in this notebook
# we plot the loss we want to optimise on th eleft (in this case: accuracy)
def plot_history(history):
  plt.figure(figsize = (12,4))
  plt.subplot(1,2,1)

  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.plot(history.epoch, np.array(history.history['accuracy']),'g-',
           label='Train accuracy')
  plt.plot(history.epoch, np.array(history.history['val_accuracy']),'r-',
           label = 'Validation accuracy')
  plt.legend()

  plt.subplot(1,2,2)
  plt.xlabel('Epoch')
  plt.ylabel('Loss minimised by model')
  plt.plot(history.epoch, np.array(history.history['loss']),'g-',
           label='Train loss')
  plt.plot(history.epoch, np.array(history.history['val_loss']),'r-',
           label = 'Validation loss')
  plt.legend()

### Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import os

#!ls '/content/gdrive/My Drive/'

### Loading and visualizing the data

In [ ]:
# load train and test data
(x_train_all, r_train_all_class), (x_test, r_test_class) = mnist.load_data()
x_train_all = x_train_all.reshape(x_train_all.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

# some preprocessing ... convert integers to floating point and rescale them to [0,1] range
x_train_all = x_train_all.astype('float32')
x_test = x_test.astype('float32')
x_train_all /= 255
x_test /= 255

print(x_train_all.shape[0], ' original train samples')
print(x_test.shape[0], ' original test samples')

# This data set contains a train set and test set
# we still need to split off a validation set

# Number of test samples
N_test = x_test.shape[0]

# split off 10000 samples for validation
N_val = 10000
N_train = x_train_all.shape[0] - N_val

# now extract the samples into train, validate and test sets
# set random state to make sure you get the same split each time
x_train, x_val, r_train_class, r_val_class = train_test_split(x_train_all, r_train_all_class, test_size = N_val, random_state=0)

# For initial explorations, it is often useful to 
# try out some things first on a smaller data set
# in comment below is the code for using all training data
# in this case, we select 10000 samples for training
# don't forget to use all training examples for your final model
# by replacing the line below by the commented one beneath it

N_train = 10000

x_train = x_train[:N_train,:]
r_train_class = r_train_class[:N_train]

print("Using ",x_train.shape[0]," train samples, ", x_val.shape[0], " validation samples and ",x_test.shape[0]," test samples")
print("Each sample has ",x_train.shape[1]," features")

In [ ]:
# The features in this data set are the pixels of a 28x28 pixel image
# You can visualise an individual image as follows
# (here for the first 9 images in the training set)

f = plt.figure(figsize=(10,10));
for idx in range(9):
    plt.subplot(3,3,idx+1)
    plt.subplots_adjust(hspace=0.5)
    plt.title("Label is " + str(r_train_class[idx]))
    plt.imshow(np.reshape(x_train[idx,:],(28,28)), cmap='Greys', interpolation='None')

# The labels are numbers from 0 to 9
print("A few labels:")
print(r_train_class[:9])

### Convert labels for multiclass

In [ ]:
r_train_all = keras.utils.to_categorical(r_train_all_class)
r_train = keras.utils.to_categorical(r_train_class)
r_val = keras.utils.to_categorical(r_val_class)
r_test = keras.utils.to_categorical(r_test_class)

# look at the new labels

## Model Architecture

In [ ]:
num_classes = 10

# this first network has 2 hidden layers
# the first layer needs to be told explicitly what the input shape is
# the output layer has 10 neurons: one neuron per class

# Note that we use the "He" initialisation scheme here, since this is often advised 
# for layers with ReLu neurons - although you are free to change this, it is not necessary for this assignment

# Also note that "dropout" is implemented in separate layers in Keras
# they are added below in comment to show you how to use them
# note that you can also start your network with a dropout layer (randomly setting input features to 0)

def initial_model():
    # create linear model
    model = Sequential()
    # we start with a first fully connected layer
    model.add(Dense(16, activation='relu', input_shape=(784,), kernel_initializer='he_uniform'))
    ## then add some dropout, set at a very low value for now
    #model.add(Dropout(0.001))
    # a second dense layer with half as many neurons
    model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
    ## some more dropout
    #model.add(Dropout(0.001))
    # and the output layer
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.001), # set to default learning rate here
                  metrics=['accuracy']) # save accuracy in addition to cross entropy error
    return model

## Training